<a href="https://colab.research.google.com/github/tangxsusan/fine_tune_bert/blob/main/3_Manning_1_HateSpeechDetection_submit3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.18.0 --quiet

     |████████████████████████████████| 4.0 MB 9.9 MB/s 
     |████████████████████████████████| 880 kB 47.2 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 596 kB 23.4 MB/s 
     |████████████████████████████████| 6.6 MB 54.5 MB/s 


In [2]:
!pip install SentencePiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade 

In [14]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import transformers
from transformers import AlbertTokenizer, AlbertModel

import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset

import pytorch_lightning as pl

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df=pd.read_csv('./drive/MyDrive/Colab Notebooks/Khilnani_LP_hate_speech_data.csv', index_col=0)
df.head()

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,""" momma said no pussy cats inside my doghouse """,0
2,"""@Addicted2Guys: -SimplyAddictedToGuys http://...",0
3,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo...",0
4,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these...",0


In [7]:
df.shape

(5593, 2)

In [8]:
def remove_username(text):
  return re.sub(r'@[\w?.!/;:]+','',text)

In [9]:
df['tweet']=df['tweet'].apply(remove_username)

In [10]:
df.head()

,tweet,class
0,!!! RT As a woman you shouldn't complain abou...,0
1,""" momma said no pussy cats inside my doghouse """,0
2,""" -SimplyAddictedToGuys http://t.co/1jL4hi8ZMF...",0
3,""" http://t.co/3gzUpfuMev"" woof woof and hot soles",0
4,""" Lemmie eat a Oreo &amp; do these dishes."" On...",0


In [11]:
train_df,test_df=train_test_split(df, test_size=0.3, random_state=42)
print(train_df.shape, test_df.shape)

(3915, 2) (1678, 2)


In [12]:
max_token_len=64

In [13]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# Create dataloaders in torch and Lightning.    
class HateSpeechDataModule_TensorDataset(pl.LightningDataModule): 
  def __init__(self, train_df, test_df, tokenizer, batch_size=8,max_token_len=64): 
    super().__init__()
    self.train_df=train_df 
    self.test_df =test_df  
    self.batch_size=batch_size
    self.tokenizer=tokenizer 
    self.max_token_len=max_token_len  


  def setup(self, stage=None):
    self.train_dataset=self.output_of_tokenizer(self.train_df)
    self.test_dataset=self.output_of_tokenizer(self.test_df)
    
  def train_dataloader(self):
    return DataLoader(self.train_dataset, shuffle=True, batch_size=self.batch_size, num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=2)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=2)

  def output_of_tokenizer(self, df):
    self.tokenized_tweet=self.tokenizer(df['tweet'].tolist(),
                              max_length=64, 
                              return_token_type_ids=True, 
                              padding='max_length', 
                              truncation=True, 
                              return_attention_mask=True,
                              return_tensors='pt')
    self.input_ids=self.tokenized_tweet['input_ids']
    self.attention_mask=self.tokenized_tweet['attention_mask']
    self.token_type_ids=self.tokenized_tweet['token_type_ids']
    self.labels=torch.tensor(df['class'].values).view(-1,1)

    return TensorDataset(self.input_ids,self.attention_mask,self.token_type_ids,self.labels)

In [16]:
# validate the dataloader
max_token_len=64
batch_size=32
dm=HateSpeechDataModule_TensorDataset(train_df, test_df,tokenizer=tokenizer,max_token_len=max_token_len,batch_size=batch_size)

In [17]:
dm.setup()
input_data=next(iter(dm.train_dataloader()))

In [20]:
len(input_data)

4

In [22]:
input_data[0].shape

torch.Size([32, 64])

In [23]:
input_data[1].shape

torch.Size([32, 64])

In [24]:
input_data[2].shape

torch.Size([32, 64])

In [25]:
input_data[3].shape

torch.Size([32, 1])

In [26]:
class HateSpeechTagger(pl.LightningModule):
  def __init__(self,n_classes: int=2, learning_rate:float=0.0001*8,
               batch_size: int=16,
               num_workers: int=4,
               **kwargs):
    super().__init__()

    self.bert=AlbertModel.from_pretrained('albert-base-v2')
    self.linear1=torch.nn.Linear(768, 768)
    self.linear2=torch.nn.Linear(768, 2)
    self.relu=torch.nn.ReLU()
    self.dropout=torch.nn.Dropout(0.5)

  def forward(self,input_ids, attention_mask, labels):
    output=self.bert(input_ids=input_ids, attention_mask=attention_mask)
    hidden_state=output[0]
    output=hidden_state[:,0]
    output=self.linear1(output)
    output=self.relu(output)
    output=self.dropout(output)
    logits=self.linear2(output)
    output=torch.nn.functional.log_softmax(logits)
    return output

  def training_step(self, batch, batch_nb):
    input_ids=batch[0]
    attention_mask=batch[1] 
    label=batch[3]
    y_hat=self(input_ids, attention_mask, label)

    loss=torch.nn.functional.nll_loss(y_hat.view(-1,2), label.view(-1).long())
    return loss

  def validation_step(self, batch, batch_nb):
    input_ids=batch[0]
    attention_mask=batch[1] 
    label=batch[3]
    y_hat=self(input_ids, attention_mask, label)
    loss=torch.nn.functional.nll_loss(y_hat.view(-1,2), label.view(-1).long())
    return loss

  def configure_optimizers(self):
    optimizer=torch.optim.Adam(self.parameters(), lr=1e-5)
    return [optimizer]

In [27]:
model=HateSpeechTagger()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
trainer=pl.Trainer(accelerator='gpu', devices=1, max_epochs=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(model,dm)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type        | Params
----------------------------------------
0 | bert    | AlbertModel | 11.7 M
1 | linear1 | Linear      | 590 K 
2 | linear2 | Linear      | 1.5 K 
3 | relu    | ReLU        | 0     
4 | dropout | Dropout     | 0     
----------------------------------------
12.3 M    Trainable params
0         Non-trainable params
12.3 M    Total params
49.103    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [30]:
model

HateSpeechTagger(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
     